In [1]:
import os
import json
import math
import price
import ratios
import requests
import momentum
import pyticker
import dividend
import questrade
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from qtrade import Questrade as qt
import pandas_datareader.data as web

In [2]:
sp500 = pyticker.get_symbols_by_index('S&P 500')
sp500

['LIN',
 'CCL',
 'SSE',
 'FTI',
 'MMM',
 'AXP',
 'AAPL',
 'BA',
 'CAT',
 'CVX',
 'CSCO',
 'KO',
 'DOW',
 'XOM',
 'GS',
 'HD',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'MCD',
 'MRK',
 'MSFT',
 'NKE',
 'PFE',
 'PG',
 'TRV',
 'UNH',
 'VZ',
 'V',
 'WMT',
 'WBA',
 'DIS',
 'ATVI',
 'ADBE',
 'AKAM',
 'ALXN',
 'GOOGL',
 'AMZN',
 'AAL',
 'AMGN',
 'ADI',
 'AMAT',
 'ADSK',
 'ADP',
 'BIIB',
 'CERN',
 'CHTR',
 'CTXS',
 'CTSH',
 'CMCSA',
 'COST',
 'CSX',
 'XRAY',
 'DISCA',
 'DISH',
 'DLTR',
 'EBAY',
 'EA',
 'EXPE',
 'FB',
 'FAST',
 'FISV',
 'GILD',
 'HSIC',
 'ILMN',
 'INCY',
 'INTU',
 'ISRG',
 'KHC',
 'LRCX',
 'MAR',
 'MXIM',
 'MCHP',
 'MU',
 'MDLZ',
 'MNST',
 'NTAP',
 'NFLX',
 'NVDA',
 'ORLY',
 'PCAR',
 'PAYX',
 'PYPL',
 'QCOM',
 'REGN',
 'ROST',
 'STX',
 'SWKS',
 'SBUX',
 'TMUS',
 'TSLA',
 'TXN',
 'TSCO',
 'ULTA',
 'VRSK',
 'VRTX',
 'WDC',
 'XLNX',
 'ABT',
 'ABBV',
 'ACN',
 'ALL',
 'MO',
 'AIG',
 'T',
 'BAC',
 'BK',
 'BRK-B',
 'BLK',
 'BKNG',
 'BMY',
 'COF',
 'C',
 'CL',
 'COP',
 'CVS',
 'DHR',
 'DUK',
 

In [ ]:
data = {'Symbol': [], 'Market_Cap (B)': []}


market_cap_threshold = 100000000000
# div_growth_threshold = 0.05
# div_yield_threshold = 0.02

count = 0

for symbol in sp500:
    count += 1
    print(f"{symbol}: {count}/{len(sp500)}")
    try:
        market_cap = ratios.calculate_market_cap(symbol)
#         div_growth = dividend.calcualte_avg_dividend_growth(symbol,10)
#         div_yield = dividend.calculate_current_dividend_yield(symbol)
    except:
        continue

    if market_cap >= market_cap_threshold:
        data['Symbol'].append(symbol)
        data['Market_Cap (B)'].append(market_cap)
        print(f"{symbol} has been added!")

data

LIN: 1/498
LIN has been added!
CCL: 2/498
SSE: 3/498
FTI: 4/498
MMM: 5/498
MMM has been added!
AXP: 6/498
AXP has been added!
AAPL: 7/498
AAPL has been added!
BA: 8/498
BA has been added!
CAT: 9/498
CAT has been added!
CVX: 10/498
CVX has been added!
CSCO: 11/498
CSCO has been added!
KO: 12/498
KO has been added!
DOW: 13/498
XOM: 14/498
XOM has been added!
GS: 15/498
GS has been added!
HD: 16/498
HD has been added!
IBM: 17/498
IBM has been added!
INTC: 18/498
INTC has been added!
JNJ: 19/498
JNJ has been added!
JPM: 20/498
JPM has been added!
MCD: 21/498
MCD has been added!
MRK: 22/498
MRK has been added!
MSFT: 23/498
MSFT has been added!
NKE: 24/498
NKE has been added!
PFE: 25/498
PFE has been added!
PG: 26/498
PG has been added!
TRV: 27/498
UNH: 28/498
UNH has been added!
VZ: 29/498
VZ has been added!
V: 30/498
V has been added!
WMT: 31/498
WMT has been added!
WBA: 32/498
DIS: 33/498
DIS has been added!
ATVI: 34/498
ADBE: 35/498
ADBE has been added!
AKAM: 36/498
ALXN: 37/498
GOOGL: 3

In [ ]:
with open('./sp500_100_billions_symbols.json', 'w') as fp:
    json.dump(data,fp)    